In [1]:
import pandas as pd

In [18]:
df = pd.read_csv('../notion_data/코스피지수.csv')

In [14]:
df.head()

,변환,원자료
0,2014-12-30,"1,915.59"
1,2015-01-02,"1,926.44"
2,2015-01-05,"1,915.75"
3,2015-01-06,"1,882.45"
4,2015-01-07,"1,883.83"


In [19]:
df = df.rename(columns={'변환' : '날짜'})

In [20]:
import pandas as pd

# df['날짜'] = pd.to_datetime(df['날짜'])
df = df.sort_values('날짜').reset_index(drop=True)

# '날짜'를 인덱스로 설정
df = df.set_index('날짜')

# ,가 포함된 숫자 문자열을 정리하고 float으로 변환
df['원자료'] = df['원자료'].astype(str).str.replace(',', '').astype(float)

# 일별 수익률 계산
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# 앞으로 볼 윈도우 끝(일)과 라벨 정의: (5일→1~5, 10일→6~10, …)
windows = [
    (5, '(1~5)'),
    (10, '(6~10)'),
    (15, '(11~15)'),
    (20, '(16~20)')
]

# 각 컬럼에 대해 미래 5일씩 네 구간별 표준편차 계산
for col in df.columns:
    for end, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(-end)           # 미래 end일만큼 앞으로(음수) 시프트
            .rolling(window=5)     # 5일 윈도우
            .std()                 # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

print(result.head())


           날짜  원자료(1~5)  원자료(6~10)  원자료(11~15)  원자료(16~20)
0  2014-12-30       NaN        NaN         NaN         NaN
1  2015-01-02       NaN        NaN         NaN         NaN
2  2015-01-05       NaN        NaN         NaN         NaN
3  2015-01-06       NaN        NaN         NaN         NaN
4  2015-01-07  0.006962   0.008811     0.00427    0.003991


C:\Users\Ph.D.Huh\AppData\Local\Temp\ipykernel_40356\2414395524.py:13: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df.pct_change()


In [21]:
result.head(30)

,날짜,원자료(1~5),원자료(6~10),원자료(11~15),원자료(16~20)
0,2014-12-30,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN
2,2015-01-05,NaN,NaN,NaN,NaN
3,2015-01-06,NaN,NaN,NaN,NaN
4,2015-01-07,0.006962,0.008811,0.004270,0.003991
5,2015-01-08,0.005405,0.008821,0.005909,0.003883
6,2015-01-09,0.005543,0.004052,0.005413,0.003856
7,2015-01-12,0.007631,0.004295,0.005340,0.004354
8,2015-01-13,0.008871,0.004406,0.003712,0.004913
9,2015-01-14,0.008811,0.004270,0.003991,0.004744


In [22]:
result = result.rename(columns={
    '원자료(-1~-5)' : '코스피수익률표준편차(1~5)',
    '원자료(-6~-10)' : '코스피수익률표준편차(6~10)',
    '원자료(-11~-15)' : '코스피수익률표준편차(11~15)',
    '원자료(-16~-20)' : '코스피수익률표준편차(16~20)',    
})

In [24]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [25]:
macro_df.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '육우(-11~-15)', '육우(-16~-20)', '미국_대두(-1~-5)',
       '미국_대두(-6~-10)', '미국_대두(-11~-15)', '미국_대두(-16~-20)',
       '코스닥수익률표준편차(-1~-5)', '코스닥수익률표준편차(-6~-10)', '코스닥수익률표준편차(-11~-15)',
       '코스닥수익률표준편차(-16~-20)'],
      dtype='object', length=160)

In [26]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [27]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '미국_대두(-11~-15)', '미국_대두(-16~-20)', '코스닥수익률표준편차(-1~-5)',
       '코스닥수익률표준편차(-6~-10)', '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)',
       '원자료(1~5)', '원자료(6~10)', '원자료(11~15)', '원자료(16~20)'],
      dtype='object', length=164)

In [ ]:
# merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)
